In [1]:
import nltk
#It contains text processing libraries for tokenization, 
#parsing, classification, stemming, tagging and semantic reasoning.
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import curses
import string
import numpy

In [2]:
import numpy as np
import tflearn

#TFlearn is a modular and transparent deep learning library built on top of Tensorflow. 
#It was designed to provide a higher-level API to TensorFlow 
#in order to facilitate and speed-up experimentations, while remaining fully transparent and compatible with it.
import tensorflow as tf
import random
tf.logging.set_verbosity(tf.logging.ERROR)

# import our chat-bot intents file
import json
#open the exact location   of CORPUS 
with open('intents.json') as jd:
    intents = json.load(jd)

W0922 14:57:32.300952 13316 deprecation_wrapper.py:119] From C:\Users\kumar\Anaconda3\lib\site-packages\tflearn\helpers\summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W0922 14:57:32.303982 13316 deprecation_wrapper.py:119] From C:\Users\kumar\Anaconda3\lib\site-packages\tflearn\helpers\trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0922 14:57:32.379739 13316 deprecation_wrapper.py:119] From C:\Users\kumar\Anaconda3\lib\site-packages\tflearn\collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0922 14:57:32.388729 13316 deprecation_wrapper.py:119] From C:\Users\kumar\Anaconda3\lib\site-packages\tflearn\config.py:123: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.

W0922 14:57:32.563499 13316 deprecation_wrapper.py:119] From C:\Users\kumar\Anaconda3\lib\site-packages\tflear

In [3]:
nltk.download('punkt')
#Punkt Sentence Tokenizer. This tokenizer divides a text into a list of sentences,
#by using an unsupervised algorithm to build a model for abbreviation words, collocations, and words that start sentences.
#It must be trained on a large collection of plaintext in the target language before it can be used.
#Create words, classes and documents
words = []
classes = []
documents = []
ignore_words = ['?','!']

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']: # PATTERN IS USER QUESTION
      
        # tokenize each word in the sentence
        
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus        # add to our classes list
        if intent['tag'] not in classes:

            documents.append((w, intent['tag']))
            classes.append(intent['tag'])


In [5]:
# stem and lower each word and remove duplicates
words = sorted(list(set([stemmer.stem(w.lower()) for w in words if w not in ignore_words])))

# remove duplicates
classes = sorted(list(set(classes)))


In [6]:
#Getting to know the documents,classes and stemmed words
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)


12 documents
12 classes ['book ticket', 'cancel', 'end', 'greeting', 'hours', 'opentoday', 'payment accept', 'pnr ', 'pnr_no ', 'refund ', 'schedule', 'thanks']
31 unique stemmed words ["'s", 'acceiv', 'ar', 'book', 'cancel', 'check', 'do', 'goodby', 'hi', 'hour', 'how', 'is', 'mod', 'my', 'no', 'of', 'op', 'pay', 'pnr', 'refund', 'schedule', 'stat', 'thank', 'the', 'ticket', 'to', 'today', 'train', 'what', 'when', 'you']


In [7]:
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)


In [8]:
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []    #bag of words which is a representaion of text that describes the occurence of wordss
    #with in a doccument
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words: #if the word appears in the pattern put 1 else 0 
        bag.append(1) if w in pattern_words else bag.append(0)
    print(bag)
    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])


[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0]
[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
print(training)

[[list([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0])
  list([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])
  list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])]
 [list([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0])
  list([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])]
 [list([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0])
  list([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])]
 [list([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0])
  list([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]
 [list([0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1])
  list([0,

In [ ]:
# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])
print(train_x,train_y)


[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 

In [ ]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 12)
net = tflearn.fully_connected(net, 12)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs') 
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1500, batch_size=8, show_metric=True)#n_epoch is the number of times network sees the data
model.save('model.tflearn')

Training Step: 2999  | total loss: 1.87047 | time: 0.004s
| Adam | epoch: 1500 | loss: 1.87047 - acc: 0.8995 -- iter: 08/12
Training Step: 3000  | total loss: 1.68418 | time: 0.006s
| Adam | epoch: 1500 | loss: 1.68418 - acc: 0.9096 -- iter: 12/12
--


In [ ]:
import pickle
pickle.dump({'words':words, 'classes':classes,'train_x':train_x,'train_y':train_y},open( "training_data", "wb" ))



# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [ ]:
# import our chat-bot intents file
import json
with open('intents.json') as jd:
    intents = json.load(jd)
    
# load our saved model
model.load('./model.tflearn')

def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

GRADIENT DESCENT ALGORITHM


Neural networks are trained iteratively using optimization techniques like gradient descent. After each cycle of training, an error metric is calculated based on the difference between prediction and target. The derivatives of this error metric are calculated and propagated back through the network using a technique called backpropagation. Each neuron’s coefficients (weights) are then adjusted relative to how much they contributed to the total error. This process is repeated iteratively until the network error drops below an acceptable threshold.

In [ ]:
def cchat():
    print("BOT: I am  your Personal Digital Assistant. What can I do for you!")
    while True:
        inp=input("YOU : ")
        if inp.lower()=="quit":
            break
        results = model.predict([bow(inp, words)])[0]
        results_index=numpy.argmax(results)
        tag=classes[results_index]
        if(results[results_index]>0.65):
            
            for tg in intents['intents']:
                if tg['tag']==tag:
                    response=tg['responses']

            print("BOT : " +random.choice(response))
        else:
            print("BOT :  I did not understand you! Try again")
cchat()

BOT: I am  your Personal Digital Assistant. What can I do for you!
YOU : hi
BOT : Hello, thanks for visiting
YOU : how to cancel a ticket
BOT : login to your account , Go to my bookings, click cancel
YOU : how to book a ticket
BOT : go to our website and click buy option
YOU : what are the payment modes
BOT : We accept VISA, Mastercard and AMEX , most major credit cards
YOU : hi
BOT : Hi there, how can I help?
YOU : quit


In [1]:
import xlsxwriter 
  
# Workbook() takes one, non-optional, argument  
# which is the filename that we want to create. 
workbook = xlsxwriter.Workbook('feedback.xlsx') 
  
# The workbook object is then used to add new  
# worksheet via the add_worksheet() method. 
worksheet = workbook.add_worksheet() 
  
# Use the worksheet object to write 
# data via the write() method. 
worksheet.write('A1', input("Enter your FEEDBACK: " )) 

  
# Finally, close the Excel file 
# via the close() method. 
workbook.close() 

Enter your FEEDBACK: thanks,you have been helpful
